In [101]:
import os 
import time
import requests  
import json  
import openai
import numpy as np
from openai import OpenAI
#from prompt_construction import read_prompt_list ,  prompt_construction module is not available here

api_key = os.environ["OPENAI_API_KEY"]  

%reload_ext autoreload
%autoreload 2

In [102]:
# Define the API endpoint URL  
def get_response_siemens_api(url, model, max_token, user_content, system_content):  
    # Define the request headers  
    headers = {  
        'accept': 'application/json',  
        'Authorization': 'Bearer {}'.format(api_key),  
        'Content-Type': 'application/json'  
    }  

    # Define the request payload  
    payload = {  
        'model': model,  
        'messages': [  
            {'role': 'user', 'content': user_content },
            {'role': 'system', 'content': system_content}           
        ],  
        'temperature': 0,  
        'stream': False,  
        'max_tokens': max_token          
    }  
      
    # Make the POST request  
    response = requests.post(url, headers=headers, data=json.dumps(payload))  
      
    # Print the response  
    return print(response.json()["choices"][0]["message"]["content"])


In [103]:
url = 'https://api.siemens.com/llm/chat/completions'  
model = 'mistral-7b-instruct'
max_token = 100
user_content = ("Please classify the given requirement into 'functional requirement' or 'non-functional requirement'. "
                "The answer should be in format {the given requirement: functional requirement or non-functional requirement}."
                "The given requirement: The system shall refresh the display every 60 seconds." )
system_content = ("You are a senior software engineer who is experienced in software requirement classification! " )

get_response_siemens_api(url, model, max_token, user_content, system_content)

 {The system shall refresh the display every 60 seconds: functional requirement}


# define useful functions

In [104]:
# Define the API endpoint URL  
def get_response_siemens_api(prompt):  
    # Define the request headers  
    headers = {  
        'accept': 'application/json',  
        'Authorization': 'Bearer {}'.format(api_key),  
        'Content-Type': 'application/json'  
    }  

    # Define the request payload  
    payload = {  
        'model': 'mistral-7b-instruct',  
        'messages': [  
            {'role': 'user', 'content': prompt },
            {'role': 'system', 'content': "You are a senior software engineer who is experienced in software requirement classification! "}           
        ],  
        'temperature': 0,  
        'stream': False,  
        'max_tokens': 12000          
    }  
      
    # Make the POST request  
    url = 'https://api.siemens.com/llm/chat/completions'  
    response = requests.post(url, headers=headers, data=json.dumps(payload))  
      
    # Print the response  
    return response.json()["choices"][0]["message"]["content"]


In [105]:
def get_completion(prompt_list, save_path):
    """load the prompt_list, get the completion from llm, and save the results."""

    start_time = time.time()  
    
    completion_list = []
    for i, prompt in enumerate(prompt_list):
        #print("The prompt: " + prompt + "\n######")
     
        j = 0
        while j < 50:
            try:   
                completion = get_response_siemens_api(prompt)
                #print(completion)
                completion_list.append(completion)
                break
        
            except: #InvalidRequestError: in case that the prompt and output reach the token limitation
                #print("{}th prompt, Connection error: try again automatically".format(i) )
                time.sleep(5)  # stop 1 second to avoid error
            j = j+1   
            
        if j == 50:
            print("{}th prompt. At least one prompt did not get the result.".format(j))
            break

        # completion = get_response_siemens_api(prompt)
        # #print(completion)
        # completion_list.append(completion)
        # break
        


    save_completion_list(save_path, completion_list)

    # calculate the execution time
    end_time = time.time()  
    print("execution time: {}".format(end_time - start_time), "second")

In [106]:
def read_prompt_list(path):
    """read the saved list of prompts 
    input
        path, string path to save the list
    """
    with open(path, 'r') as file:  
        content = file.read()  
    prompt_list_read = content.split('\n\n\n')   

    return prompt_list_read


In [107]:
def save_completion_list(path, completion_list):
    """save the constructed prompts with few-shot examples in a list 
    input
        path, string path to save the list
        prompt_list, list of constructed prompts from the first requirement/sentence in test dataset to the last one
    """
    with open(path, 'w', newline='\n') as file:  
        for i, completion in enumerate(completion_list):
            if i+1 == len(completion_list):
                file.write(completion)
            else:     
                file.write(completion + "\n\n\n")

# get completion on promise dataset

## binary classification

In [53]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_0_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_0_bi.txt"
get_completion(prompt_list, save_path)

execution time: 109.59332823753357 second


In [54]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_5_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_5_bi.txt"
get_completion(prompt_list, save_path)

execution time: 123.91648483276367 second


In [55]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_10_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_10_bi.txt"
get_completion(prompt_list, save_path)

execution time: 119.93200516700745 second


In [56]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_20_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_20_bi.txt"
get_completion(prompt_list, save_path)

execution time: 122.57221627235413 second


In [57]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_40_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_40_bi.txt"
get_completion(prompt_list, save_path)

execution time: 121.00452280044556 second


In [58]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_80_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_80_bi.txt"
get_completion(prompt_list, save_path)

execution time: 122.31055903434753 second


In [108]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_160_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_160_bi.txt"
get_completion(prompt_list, save_path)

execution time: 122.77507400512695 second


#### binary embedding

In [59]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_0_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_0_bi.txt"
get_completion(prompt_list, save_path)

execution time: 117.28138899803162 second


In [60]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_5_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_5_bi.txt"
get_completion(prompt_list, save_path)

execution time: 124.36202907562256 second


In [61]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_10_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_10_bi.txt"
get_completion(prompt_list, save_path)

execution time: 122.3440887928009 second


In [62]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_20_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_20_bi.txt"
get_completion(prompt_list, save_path)

execution time: 122.39480924606323 second


In [63]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_40_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_40_bi.txt"
get_completion(prompt_list, save_path)

execution time: 124.34133410453796 second


In [64]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_80_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_80_bi.txt"
get_completion(prompt_list, save_path)

execution time: 128.29276990890503 second


In [109]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_160_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_160_bi.txt"
get_completion(prompt_list, save_path)

execution time: 137.29553890228271 second


#### binary tfidf

In [91]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_0_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_0_bi.txt"
get_completion(prompt_list, save_path)

execution time: 115.91063594818115 second


In [92]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_5_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_5_bi.txt"
get_completion(prompt_list, save_path)

execution time: 121.88522005081177 second


In [93]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_10_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_10_bi.txt"
get_completion(prompt_list, save_path)

execution time: 122.12191200256348 second


In [94]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_20_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_20_bi.txt"
get_completion(prompt_list, save_path)

execution time: 122.25008893013 second


In [95]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_40_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_40_bi.txt"
get_completion(prompt_list, save_path)

execution time: 120.33709502220154 second


In [96]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_80_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_80_bi.txt"
get_completion(prompt_list, save_path)

execution time: 129.56676816940308 second


In [110]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_160_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_160_bi.txt"
get_completion(prompt_list, save_path)

execution time: 127.41378712654114 second


## multi-class classification

#### promise multi-class random

In [71]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_0.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_0.txt"
get_completion(prompt_list, save_path)

execution time: 103.16644406318665 second


In [72]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_5.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_5.txt"
get_completion(prompt_list, save_path)

execution time: 123.59215784072876 second


In [73]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_10.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_10.txt"
get_completion(prompt_list, save_path)

execution time: 151.8356590270996 second


In [74]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_20.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_20.txt"
get_completion(prompt_list, save_path)

execution time: 124.51837396621704 second


In [75]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_40.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_40.txt"
get_completion(prompt_list, save_path)

execution time: 122.07912182807922 second


In [76]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_80.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_80.txt"
get_completion(prompt_list, save_path)

execution time: 120.85491275787354 second


In [111]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_160.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_160.txt"
get_completion(prompt_list, save_path)

execution time: 123.72332906723022 second


#### promise multiclass embedding

In [77]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_0.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_0.txt"
get_completion(prompt_list, save_path)

execution time: 121.12382888793945 second


In [78]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_5.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_5.txt"
get_completion(prompt_list, save_path)

execution time: 121.81056785583496 second


In [79]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_10.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_10.txt"
get_completion(prompt_list, save_path)

execution time: 117.41837573051453 second


In [80]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_20.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_20.txt"
get_completion(prompt_list, save_path)

execution time: 125.62616109848022 second


In [81]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_40.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_40.txt"
get_completion(prompt_list, save_path)

execution time: 123.28952097892761 second


In [82]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_80.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_80.txt"
get_completion(prompt_list, save_path)

execution time: 125.4775390625 second


In [112]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_160.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_160.txt"
get_completion(prompt_list, save_path)

execution time: 134.59514784812927 second


#### promise multiclass tf-idf

In [83]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_0.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_0.txt"
get_completion(prompt_list, save_path)

execution time: 112.45754504203796 second


In [84]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_5.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_5.txt"
get_completion(prompt_list, save_path)

execution time: 120.38309717178345 second


In [85]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_10.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_10.txt"
get_completion(prompt_list, save_path)

execution time: 125.04896903038025 second


In [86]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_20.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_20.txt"
get_completion(prompt_list, save_path)

execution time: 115.85242700576782 second


In [87]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_40.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_40.txt"
get_completion(prompt_list, save_path)

execution time: 126.12441611289978 second


In [88]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_80.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_80.txt"
get_completion(prompt_list, save_path)

execution time: 133.32836818695068 second


In [113]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_160.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_160.txt"
get_completion(prompt_list, save_path)

execution time: 127.57248735427856 second


# get completion on nfr dataset

## multi-class classification

#### nfr multi_class random

In [89]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_random_0.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_random_0.txt"
get_completion(prompt_list, save_path)

execution time: 3460.546215057373 second


In [97]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_random_5.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_random_5.txt"
get_completion(prompt_list, save_path)

execution time: 3478.666871070862 second


In [114]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_random_10.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_random_10.txt"
get_completion(prompt_list, save_path)

execution time: 3468.873160123825 second


In [115]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_random_20.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_random_20.txt"
get_completion(prompt_list, save_path)

execution time: 3486.1135108470917 second


In [116]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_random_40.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_random_40.txt"
get_completion(prompt_list, save_path)

execution time: 3484.3390600681305 second


In [117]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_random_80.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_random_80.txt"
get_completion(prompt_list, save_path)

execution time: 3487.0319340229034 second


In [118]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_random_160.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_random_160.txt"
get_completion(prompt_list, save_path)

execution time: 27610.913210868835 second


#### nfr multi_class embedding

In [119]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_embedding_0.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_embedding_0.txt"
get_completion(prompt_list, save_path)

execution time: 9217.068017959595 second


In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_embedding_5.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_embedding_5.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_embedding_10.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_embedding_10.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_embedding_20.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_embedding_20.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_embedding_40.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_embedding_40.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_embedding_80.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_embedding_80.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_embedding_160.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_embedding_160.txt"
get_completion(prompt_list, save_path)

#### nfr multi_class tfidf

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_tfidf_0.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_tfidf_0.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_tfidf_5.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_tfidf_5.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_tfidf_10.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_tfidf_10.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_tfidf_20.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_tfidf_20.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_tfidf_40.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_tfidf_40.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_tfidf_80.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_tfidf_80.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_tfidf_160.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_tfidf_160.txt"
get_completion(prompt_list, save_path)